In [1]:
import os
import time
import numpy as np
import random as rnd

In [2]:
#Instances in each generation
n_population = 100

#Iterations
n_generation = 100

# Mutation Chance
prob_mutation = 0.8

# Crossover Chance
prob_crossover = 0.8

In [3]:
def get_best(generation,distance_matrix,n_cities):
    
    new_generation = []
    
    prob, fit, dist = fitness(generation,distance_matrix)
    
    gene = (generation[0],dist[0],fit[0])
    
    for i in range(n_population):
    
        #Find Parents
        parents = selection(generation,prob,fit,dist)
        #Get Child
        child = get_child(parents,distance_matrix,n_cities)
       
        # Check if the CHILD formed is better
        if child[2] > gene[2]:
            gene = child
            
        new_generation.append(child[0])
    
    #Generation = [Permutation of Numbers]
    #Gene = [Gene, Distance, Fitness]
    return generation, gene
        

In [4]:
def selection(generation,prob,fitness,distance):
    
    parent = []
    
    rand_prob = rnd.random()
    
#     bprob = probability[0]
#     sbprob = probability[1]
    
    parent.append(generation[0])
    parent.append(generation[1])
    
    parent1=False
    
    #Iterate over all Population
    for i in range(0,len(generation)):
        
        if prob[i] > rand_prob and parent1 == False:
            parent[0] = (generation[i],distance[i],fitness[i])
            parent1 = True
            
        elif  prob[i] > rand_prob:
            parent[1] = (generation[i],distance[i],fitness[i])
            break
    
    
#     #Iterate over all Population
#     for i in range(1,len(generation)):
        
#         if bprob < probability[i]:
#             sbprob = bprob
#             bprob = probability[i]
#             parent[1] = parent[0]
#             parent[0] = (generation[i],distance[i],fitness[i])
            
#         elif sbprob < probability[i]:
#             sbprob = probability[i]
#             parent[1] = (generation[i],distance[i],fitness[i])
    
    #Parent  = [Gene, Distance, Fitness]
    return parent

In [5]:
def get_child(parent,distance_matrix,n_cities):
    
    x = rnd.random()
    
    if x < prob_crossover:   
        m_child = crossover(parent,n_cities)
    else:
        m_child = parent[0][0]
    
    y = rnd.random()
    
    if y < prob_mutation:
        xchild = mutate(m_child)
    else:
        x_child = m_child
    
    fit,dist = calc_fitness(xchild,distance_matrix)
    
    child = (gene, dist, fit)
    
    #Child  = [Gene, Distance, Fitness]
    return child

In [6]:
def crossover(parent,n_cities):
    
#     n_cities = len(parent[0][0])
    
    child = np.zeros(n_cities)
    city_used = dict()
    
    startx = rnd.randrange(0,n_cities)
    endx = rnd.randrange(startx,n_cities)
    
    for i in range(1,n_cities+1):
        city_used[i] = False
    
    for i in range(startx,endx):
        child[i] = parent[0][0][i]
        city_used[child[i]] = True
    
    end = 0
    
    for i in range(endx,n_cities):
        for j in range(end,n_cities):
            if city_used[parent[1][0][j]] == False:
                child[i] = parent[1][0][j]
                city_used[child[i]] = True
                end = j+1
                break
                
    for i in range(startx):
        for j in range(end,n_cities):
            if city_used[parent[1][1][j]] == False:
                child[i] = parent[1][1][j]
                city_used[child[i]] = True
                break
        
    #child  = [Gene]
    return child

In [7]:
def mutate(m_child):
    
    startx = rnd.randrange(0,n_cities)
    endx = rnd.randrange(startx,n_cities)
    
    temp = m_child[endx]
    m_child[endx] = child[startx]
    m_child[startx] = temp
    
    #m_child  = Gene
    return child

In [8]:
def encoding(n_cities):
    
    generation = []
    gene = []
    
    #enc_gen = dict(False)
    
    for i in range(n_population):
        gene =  np.random.permutation(n_cities)
        generation.append(gene)
            
    #Generation = [Permutation of Numbers]
    return generation

In [9]:
def calc_fitness(gene,distance_matrix):
    
    dist = 0.0
    
    for i in range(len(gene)-1):
        dist += distance_matrix[gene[i]][gene[i+1]]
    
    fitness = 1/dist
    
    #Return Fitness and Distance of a Gene
    return fitness,dist

In [10]:
def fitness(generation,distance_matrix):
    
    s_fitness = 0
    
    prob = dict()  
    distance = dict()
    fitness=dict()
    
    i=0
    
    for gene in generation:
        fit,dist = calc_fitness(gene,distance_matrix)
        s_fitness += fit
        #prob[i] = fit
        fitness[i] = fit
        distance[i] = dist
        i+=1
    
    i=0
    
    for gene in generation:
        prob[i] = fitness[i]/s_fitness
        i+=1
    
    #Return Probability, Fitness, Distance of full generation
    return prob,fitness,distance

In [33]:
def read_data(filename):
    
    file = open(filename,"r")
    lines = file.readlines()
    
    data_type = lines[0]
    n_cities = int(lines[1])
    
    cities = dict()

    for i in range(2,n_cities+2): 
        x = lines[i].split("\n")
        y = x[0].split(" ")

        #cities[i-2].append((i-1,y))
        cities[i-2] = y

    distance_matrix = np.zeros((n_cities,n_cities))

    for i in range(n_cities+2,2*n_cities+2):
        x = lines[i].split("\n")
        y = x[0].split(" ")

        for j in range(0,n_cities):
            distance_matrix[i-n_cities-2][j] = float(y[j])

    print(distance_matrix)

    return n_cities, cities, distance_matrix   

    

In [34]:
def main():
    
    dataset=["euc_100","euc_250","euc_500","noneuc_100","noneuc_250","noneuc_500"]
    
    n_cities, cities, distance_matrix = read_data("abc")
    
    #First Generation
    generation = encoding(n_cities)
    
    best_gene = (generation[0],0.0,0.0)
      
#     for i in range(n_generation):
        
#         generation, gene =  get_best(generation,distance_matrix,n_cities)
        
#         if gene[2] > best_gene[2]:
#             best_gene = gene
        
#         print("Distance = ",best_gene[1])
    
#     print("Distance = ",best_gene[1])

In [35]:
# if __name__ == "__main__":
main()

[[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]]
